In [1]:
from sklearn.datasets import fetch_openml #sklearn to download data
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
type(mnist)

sklearn.utils.Bunch

In [3]:
X, y = mnist['data'], mnist['target']
print(X.shape)
print(y.shape)
print(type(X))
from sklearn.model_selection import cross_val_score


(70000, 784)
(70000,)
<class 'numpy.ndarray'>


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,  y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

svm_clf = OneVsRestClassifier(SVC(gamma="scale"))

svm_clf = SVC()
svm_clf.fit(X_train_scaled[:4000], y_train[:4000])

SVC()

In [16]:
from sklearn.metrics import accuracy_score
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9288571428571428

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, 
                                   verbose=2,cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled[:2000], y_train[:2000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   0.4s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   0.4s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=   0.4s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   0.4s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   0.4s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=   0.4s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   0.3s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   0.3s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   0.3s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   0.4s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=   0.4s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000028FA0A24E50>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000028F9F795AC0>},
                   random_state=42, verbose=2)

In [19]:
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=2.560186404424365, gamma=0.002051110418843397)

In [20]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.997625

In [21]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9665714285714285